# Day 7

[https://adventofcode.com/2022/day/7]()

In [266]:
from typing import List
import re

In [267]:
with open('./inputs/day07_input_full.txt') as file:
    lines: List = [line.rstrip().split(',') for line in file]
    lines = [l[0] for l in lines]

In [268]:
class Node:
    """
    Represents a node in the file system hierarchy. This can be a directory or a file.
    """
    parent = None
    name: str = None
    type: None
    size: int = None
    children = []

    def __init__(self, parent, name, type='dir', size=0):
        self.parent = parent
        self.name = name
        self.type = type
        self.size = size
        self.children = []

    def __repr__(self):
        return f'Node(name={self.name}, type={self.type}, parent={None if self.parent is None else self.parent.name}, size={self.size}, child_count={len(self.children)})'

    def add_child(self, node):
        self.children.append(node)
        node.parent = self

    def has_children(self):
        return len(self.children) > 0

In [269]:
def parse(ptr: Node, line: str):
    """ Parse a line in the command output"""

    # Match 'cd' command
    m = re.match('^\$ (cd) (.+)', line)
    if m is not None:
        if m.group(2) == '..':
            # move one level up the tree
            return ptr.parent
        else:
            # when changing to a dir, create a new node to represent it and add to the current pointer as a child
            new_dir = Node(None if m.group(2) == '/' else ptr, name=m.group(2))
            ptr.add_child(new_dir)
            return new_dir

    # Match files in the 'ls' output. Note that I'm ignoring dir lines from the 'ls' output as I create the dir node at 'cd' (above).
    m = re.match('^(\d+) (.+)', line)
    if m is not None:
        new_file = Node(ptr, name=m.group(2), type='file', size=int(m.group(1)))
        ptr.add_child(new_file)

    return ptr

In [270]:
wrapper = Node(None, None)
pointer = wrapper
for l in lines:
    pointer = parse(pointer, l)

In [271]:
def calc_dir_size(node):
    """ Traverse the filesystem hierarchy depth-first and calculate the size of the containing directory as it traverse back up"""
    if node.has_children():
        for c in node.children:
            node.size += calc_dir_size(c)
    return node.size

In [272]:
# Ignore 'start', which is a wrapper node. What I need it the root.
root = wrapper.children[0]

In [273]:
calc_dir_size(root)

40572957

In [274]:
# Check the root if the totals match with what's in the quiz
root

Node(name=/, type=dir, parent=None, size=40572957, child_count=9)

In [275]:
def filter_nodes(node, pred):
    """ Select nodes in the filesystem hierarchy that satisfies a given predicate"""
    bucket = []
    if node.has_children():
        for c in node.children:
            bucket += filter_nodes(c, pred)
    return ([node] if pred(node) else []) + bucket


### Answer 1

In [276]:
def max_100000(c):
    """ A predicate to filter the directories with its total size at most 100000"""
    return c.type == 'dir' and c.size <= 100000

In [277]:
result = filter_nodes(root, max_100000)
sum([r.size for r in result])

1447046

### Answer 2

In [278]:
total_disk_size = 70000000
unused_space_required = 30000000
currently_used = root.size
currently_used

40572957

In [279]:
currently_free = total_disk_size - currently_used
currently_free

29427043

In [280]:
need_to_reclaim = unused_space_required - currently_free
need_to_reclaim

572957

In [281]:
result = filter_nodes(root, lambda node: node.type == 'dir')
sorted([n.size for n in (filter(lambda e: e.size >= need_to_reclaim, result))])[0]

578710

### Submission

Submitted answers:
* Answer 1 = 1447046
* Answer 2 = 578710